In [60]:
import pandas as pd
dfg = pd.read_csv ('mike_courses.csv')
print(dfg)

                                  Course          City           State  \
0                            Otter Creek      Columbus         Indiana   
1                         Sterling Farms      Stamford     Connecticut   
2                              Oak Hills       Norwalk     Connecticut   
3                             Centennial        Carmel        New York   
4                            Great River       Milford     Connecticut   
..                                   ...           ...             ...   
136  The Legends Golf Resort: Heathlands  Mrytle Beach  South Carolina   
137            Grand Dunes Resort Course  Mrytle Beach  South Carolina   
138                Beachwood Golf Course  Mrytle Beach  South Carolina   
139             Apple Greens Golf Course      Highland        New York   
140                   Mohonk Golf Course     New Paltz        New York   

    Country  Year  
0        US   NaN  
1        US   NaN  
2        US   NaN  
3        US   NaN  
4        US

In [61]:
import sqlite3
cnx = sqlite3.connect('garmin.db')

df_db = pd.read_sql_query("SELECT * FROM courses", cnx)
print(df_db.head())

   id                   g_course               g_address          g_city  \
0   1                  BAD MATCH                 XXXXXXX         XXXXXXX   
1   2       Big Spring Lake Golf     860 Country Club Rd     Albertville   
2   3   Willow Brook Golf Course              134 Par Dr     Albertville   
3   4  Willow Point Country Club    2544 Willow Point Rd  Alexander City   
4   5    Aliceville Country Club          285 Fairway Ln      Aliceville   

   g_state g_country  g_latitude  g_longitude  
0       XX        XX   30.000000   -50.000000  
1  Alabama        US   34.297449   -86.199218  
2  Alabama        US   34.352559   -86.119746  
3  Alabama        US   32.786363   -85.976556  
4  Alabama        US   33.303987   -88.058330  


In [ ]:
import fuzzymatcher
cnx = sqlite3.connect('garmin.db')

df_all_courses = pd.read_sql_query("SELECT * FROM courses", cnx)

df_george_raw = pd.read_csv ('george_courses.csv')

df_mike_raw = pd.read_csv ('mike_courses.csv')


g_left = ["Course", "City", "State", "Country"]
db_right = ["g_course", "g_city", "g_state", "g_country"]
fuzzy_match_george = fuzzymatcher.fuzzy_left_join(df_george_raw, df_all_courses, g_left, db_right)
fuzzy_match_george.to_csv("george_fuzzy.csv")
fuzzy_match_george

In [59]:
lt = fuzzymatcher.link_table(dfg, df_db, g_left, db_right)
lt
lt.to_csv("mike_link.csv")

In [ ]:
lt2 = lt.loc[[3]]
lt2

In [ ]:
lt.drop(lt.index.to_list()[1:] ,axis = 0 )

In [ ]:
lt2 = fuzzymatcher.link_table(dfg.loc[[2]], df_db, g_left, db_right).head(5)
lt2

In [69]:
cnx = sqlite3.connect('garmin.db')

df = pd.read_sql_query("SELECT * FROM users WHERE good_match=0", cnx)
cnx = sqlite3.connect('garmin.db')

df_all_courses = pd.read_sql_query("SELECT * FROM courses", cnx)
df_bad_match = pd.read_sql_query("SELECT * FROM users WHERE good_match=0", cnx)
bad_left = ["course", "city", "state", "country"]
db_right = ["g_course", "g_city", "g_state", "g_country"]
bad_match_table = fuzzymatcher.link_table(df_bad_match, df_all_courses, bad_left, db_right)
bad_match_table.to_csv("bad_match_table.csv")
bad_match_table

,__id_left,__id_right,match_score,match_rank,course,g_course,city,g_city,state,g_state,country,g_country
0,0_left,276_right,0.002038,1,University of Alabama Golf Course,Country Club of Tuscaloosa,Tuscaloosa,Tuscaloosa,Alabama,Alabama,US,US
1,0_left,280_right,-0.196563,2,University of Alabama Golf Course,Tall Pines Golf Club,Tuscaloosa,Tuscaloosa,Alabama,Alabama,US,US
2,0_left,277_right,-0.252487,3,University of Alabama Golf Course,Indian Hills Country Club,Tuscaloosa,Tuscaloosa,Alabama,Alabama,US,US
3,0_left,24_right,-0.329903,4,University of Alabama Golf Course,Auburn University Club,Tuscaloosa,Auburn,Alabama,Alabama,US,US
4,0_left,278_right,-0.330866,5,University of Alabama Golf Course,NorthRiver Yacht Club,Tuscaloosa,Tuscaloosa,Alabama,Alabama,US,US
...,...,...,...,...,...,...,...,...,...,...,...,...
3001,51_left,15498_right,-0.056282,4,Grand Dunes Resort Course,Possum Trot Golf Course,Mrytle Beach,North Myrtle Beach,South Carolina,South Carolina,US,US
3002,51_left,15465_right,-0.076009,5,Grand Dunes Resort Course,Man O' War Golf Course,Mrytle Beach,Myrtle Beach,South Carolina,South Carolina,US,US
3003,51_left,15469_right,-0.088865,6,Grand Dunes Resort Course,Myrtle Beach National Golf Club - West Course,Mrytle Beach,Myrtle Beach,South Carolina,South Carolina,US,US
3004,51_left,15468_right,-0.156687,7,Grand Dunes Resort Course,Myrtle Beach National Golf Club - South Creek ...,Mrytle Beach,Myrtle Beach,South Carolina,South Carolina,US,US


In [75]:
df_george_raw = pd.read_csv ('george_courses.csv')

df_mike_raw = pd.read_csv ('mike_courses.csv')
df_george_raw

,Course,City,State,Country,Year
0,Wynlakes,Montgomery,Alabama,US,1990
1,Lagoon Park,Montgomery,Alabama,US,1996
2,University of Alabama Golf Course,Tuscaloosa,Alabama,US,1997
3,Hidden Meadows,Nortport,Alabama,US,1998
4,Bent Brook,Bessemer,Alabama,US,1999
...,...,...,...,...,...
154,Gateway,Montgomery,Alabama,US,2022
155,Mohansic Golf Course,Yorktown Heights,New York,US,2022
156,Dead Horse Lake Golf Course,Knoxville,Tennessee,US,2022
157,Farm Golf Club,Rocky Face,Georgia,US,2022


In [79]:
l = df_george_raw.append(df_mike_raw)
l

C:\Users\George\AppData\Local\Temp\ipykernel_11588\3445217143.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l = df_george_raw.append(df_mike_raw)


,Course,City,State,Country,Year
0,Wynlakes,Montgomery,Alabama,US,1990.0
1,Lagoon Park,Montgomery,Alabama,US,1996.0
2,University of Alabama Golf Course,Tuscaloosa,Alabama,US,1997.0
3,Hidden Meadows,Nortport,Alabama,US,1998.0
4,Bent Brook,Bessemer,Alabama,US,1999.0
...,...,...,...,...,...
136,The Legends Golf Resort: Heathlands,Mrytle Beach,South Carolina,US,NaN
137,Grand Dunes Resort Course,Mrytle Beach,South Carolina,US,NaN
138,Beachwood Golf Course,Mrytle Beach,South Carolina,US,NaN
139,Apple Greens Golf Course,Highland,New York,US,NaN


In [101]:
cnx = sqlite3.connect('garmin.db')

df_all_courses = pd.read_sql_query("SELECT * FROM courses", cnx)

df_george_raw = pd.read_csv ('george_courses.csv')

df_mike_raw = pd.read_csv ('mike_courses.csv')
df_all = pd.read_sql_query("SELECT * FROM users", cnx)
df_all = df_all.drop(df_all.columns[[7,10,11,12,13,14,15,16]], axis=1)
df_all

g_left = ["course", "city", "state", "country"]
db_right = ["g_course", "g_city", "g_state", "g_country"]

fuzzy_table = fuzzymatcher.link_table(df_all, df_all_courses, g_left, db_right)
fuzzy_table = fuzzy_table[fuzzy_table['match_rank'] < 6]
fuzzy_table.to_csv("fuzzy_table.csv")

In [81]:
cnx = sqlite3.connect('garmin.db')

df_all_courses = pd.read_sql_query("SELECT * FROM courses", cnx)

df_george_raw = pd.read_csv ('george_courses.csv')

df_mike_raw = pd.read_csv ('mike_courses.csv')
df_all_users = pd.read_sql_query("SELECT * FROM users", cnx)


g_left = ["course", "city", "state", "country"]
db_right = ["g_course", "g_city", "g_state", "g_country"]

l.to_csv("joined.csv")
fuzzy_table = fuzzymatcher.link_table(df_george_raw, df_all_courses, g_left, db_right)
fuzzy_table = fuzzy_table[fuzzy_table['match_rank'] < 6]
fuzzy_table.to_csv("fuzzy_table.csv")

C:\Users\George\AppData\Local\Temp\ipykernel_11588\1819066020.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l=df_george_raw.append(df_mike_raw)
